In [78]:
import torch
import data_prepro

In [79]:
device='cpu'

In [80]:
from torch.nn.utils import weight_norm
import torch.nn as nn

In [81]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        # chomp_size(padding size): (kernel_size-1) * dilation_size 
        self.chomp_size = chomp_size  

    def forward(self, x):
        # output: (N, C_in, L_in - chomp_size)
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        #--------------------- Dilated Causal Convolution --------------------- 
        '''
        input: (N, C_in, L_in) -> (N, C_out, L_in)
        output sequence의 길이는 변하지 않는다. 
        '''
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        #----------------------------------------------------------------------
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        #--------------------- Dilated Causal Convolution --------------------- 
        '''
        input: (N, C_in, L_in) -> output: (N, C_out, L_in)
        output sequence의 길이는 변하지 않는다. 
        '''
        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        #----------------------------------------------------------------------
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)
        
        # Dilated Causal Conv -> WeightedNorm -> ReLU -> Dropout -> ... (논문의 Residual block 구조와 동일)
        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        
        # Residual Connections 
        '''  
        만약 Dilated Causal Conv를 적용하기 전의 input channel과 적용한 후의 
        output channel이 달라질 경우를 대비하여 추가적인 1*1 convolution을 추가해줌.

        input: (N, C_in, L_in) -> output: (N, C_out, L_in)
        output sequence의 길이는 변하지 않는다. (1*1 convolution)
        '''  
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            # dilation factor는 layer의 깊이에 지수적으로 증가 (2^i, i: layer depth) 
            dilation_size = 2 ** i 
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            # 설정된 layer의 깊이만큼 TemporalBlock 생성 
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)


class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size=kernel_size, dropout=dropout)
        self.linear = nn.Linear(num_channels[-1], output_size)

    def forward(self, inputs):
        """
        Inputs have to have dimension (N, C_in, L_in)
        Flatten된 MNIST의 경우에는 (batch_size, 1, 784)
        """
        y1 = self.tcn(inputs)  # input should have dimension (N, C, L)
        o = self.linear(y1[:, :, -1]) # sequence의 마지막 time step로 linear계산 -> 분류예측
        return o

In [82]:
num_epochs = 300 #1000 epochs
learning_rate = 0.01 #0.001 lr

kernel_size = 5
stride = 1
dilation = 1
padding=2
dropout=0.2
input_size = 24 #number of features
hidden_size = 256 #number of features in hidden state
num_channels = [512,512,256] #number of stacked lstm layers

output_size = 1 #number of output classes 

In [83]:
device = torch.device('cpu')
model = TCN(input_size, output_size, num_channels, kernel_size, dropout)
model.load_state_dict(torch.load('tcn_600epochs.pt', map_location=device))

<All keys matched successfully>

In [84]:
train,test = data_prepro.loadData('datasets/data_tr_city.csv','datasets/data_ts_city.csv',trainYearRange1=2019,trainYearRange2=2020)

In [85]:
X,Y = data_prepro.XDataToXAndYSeq(train)

In [86]:
X = X.reshape(-1,)

In [87]:
X[-24:].reshape(1,24,1).shape

(1, 24, 1)

In [88]:
X = X.tolist()

In [89]:
Y_pred = []

In [90]:
for i in range(365*24):
    pred = model(torch.Tensor(X[-24:]).reshape(1,24,1)).tolist()[0][0]
    X.append(pred)
    Y_pred.append(pred)

In [91]:
X[-24:]

[-0.04986804351210594,
 -0.044238969683647156,
 -0.047725457698106766,
 -0.04425707459449768,
 -0.04495525732636452,
 -0.04756781831383705,
 -0.046754173934459686,
 -0.05200258269906044,
 -0.03776133805513382,
 -0.04676598310470581,
 -0.046376731246709824,
 -0.0525997020304203,
 -0.04966050013899803,
 -0.04052700847387314,
 -0.04829110950231552,
 -0.04758134111762047,
 -0.04097092151641846,
 -0.04878951236605644,
 -0.04737686738371849,
 -0.046871695667505264,
 -0.0457451269030571,
 -0.04775553196668625,
 -0.04460683837532997,
 -0.04405198618769646]

In [92]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8424 entries, 0 to 8423
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   datetime             8424 non-null   datetime64[ns]
 1   구미 혁신도시배수지 유출유량 적산차  8424 non-null   float64       
 2   year                 8424 non-null   int64         
 3   month                8424 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 263.4 KB
